In [119]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind, f, f_oneway, chi2_contingency, ttest_1samp

In [120]:
def get_data_from_sheets(excel_file):
    return {sheet_name: excel_file.parse(sheet_name) 
          for sheet_name in excel_file.sheet_names}

In [121]:
index_file_path = '../Data/Index_Data.xlsx'

index_data = pd.ExcelFile(index_file_path)

index_sheets = get_data_from_sheets(index_data)



MF_file_path = '../Data/Mutual_Funds_Data.xlsx'

MF_data = pd.ExcelFile(MF_file_path)

MF_sheets = get_data_from_sheets(MF_data)



GB_file_path = '../Data/Gold_Bond_Data.xlsx'

GB_data = pd.ExcelFile(GB_file_path)

GB_sheets = get_data_from_sheets(GB_data)

In [122]:
len(index_sheets['NSEI'])

4263

In [123]:
index_sheets['NSEI'].isnull().sum().sum()

1

## 1. Two Sample T-test

### Null Hypothesis: The mean closing price of `NSEI` is equal to `BSESN`.
### Alternative Hypothesis: The mean closing price of `NSEI` is not equal to `BSESN`.

In [124]:
NSEI = index_sheets['NSEI']
BSESN = index_sheets['BSESN']

In [125]:
print(NSEI['Close'].mean())
print(BSESN['Close'].mean())

10189.347246485475
23414.174334683583


In [126]:
t_stat, test1_p_val = ttest_ind(NSEI['Close'], BSESN['Close'])
print(f"T-Statistic: {t_stat}, p-value: {test1_p_val}")

T-Statistic: -41.71253300179125, p-value: 0.0


In [127]:
def test_hypothesis(p_val):
    if p_val < 0.05:
        return "Reject the null hypothesis."
    else:
        return "Fail to reject the null hypothesis."

In [128]:
test_hypothesis(test1_p_val)

'Reject the null hypothesis.'

#### Here, we have rejected the null hypothesis.
#### The mean closing price of `NSEI` is not equal to `BSESN`.

## 2. F-Test

### Null Hypothesis: The variances of returns of `CNXIT` and `CNXAUTO` are equal.
### Alternative Hypothesis: The variances of returns of `CNXIT` and `CNXAUTO` are not equal.

In [129]:
CNXIT = index_sheets['CNXIT']
CNXAUTO = index_sheets['CNXAUTO']

In [130]:
CNXIT.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock_Splits', 'Percent_Change_In_Price'],
      dtype='object')

In [131]:
CNXIT_variance = np.var(CNXIT['Percent_Change_In_Price'], ddof=1)
CNXAUTO_variance = np.var(CNXAUTO['Percent_Change_In_Price'], ddof=1)

f_value = CNXIT_variance / CNXAUTO_variance

df1 = len(CNXIT['Percent_Change_In_Price']) - 1
df2 = len(CNXAUTO['Percent_Change_In_Price']) - 1

p_left = f.cdf(f_value, df1, df2) # cumulative distribution function (CDF)
p_right = f.sf(f_value, df1, df2) # Survival function = 1 - cdf
test2_p_val = 2 * min(p_left, p_right)

print('Degree of freedom 1:',df1)
print('Degree of freedom 2:',df2)
print("F-statistic:", f_value)
print("p_left:", p_left, "p_right:", p_right)
print("p-value:", test2_p_val)

Degree of freedom 1: 3973
Degree of freedom 2: 3325
F-statistic: 94.5316845231711
p_left: 0.9999999999999999 p_right: 0.0
p-value: 0.0


In [132]:
test_hypothesis(test2_p_val)

'Reject the null hypothesis.'

#### Here, reject the null hypothesis.

#### It means that The variances of returns of `CNXIT` and `CNXAUTO` are not equal

## 3. One-way ANOVA

### Null Hypothesis: The mean returns of all mutual funds are the same.
### Alternative Hypothesis: At least one mutual fund has a different mean return.

In [148]:
MF_mean_return = [round((df['Percent_Change_In_Price'].dropna().mean() / 100), 4) for df in MF_sheets.values()]
MF_mean_return

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0003,
 0.0]

In [134]:
f_stat, test3_p_val = f_oneway(
    *[df['Percent_Change_In_Price'].dropna() for df in MF_sheets.values()]
)
print(f"F-Statistic: {f_stat}, p-value: {test3_p_val}")

F-Statistic: 0.44315690521568885, p-value: 0.9755151375252552


In [135]:
test_hypothesis(test3_p_val)

'Fail to reject the null hypothesis.'

#### Here. we fail to reject the null hypothesis.

#### Because of not having sufficient evidences, we are not able to prove that At least one mutual fund has a different mean return.

## 4. Chi-Square Test

### Null Hypothesis: Gold bond returns are independent of market conditions (e.g., bearish or bullish).
### Alternative Hypothesis: Gold bond returns are not independent of market conditions.

In [136]:
GoldBond = GB_sheets['Gold Bond']

In [137]:
GoldBond['Gold_Return_Category'] = GoldBond['Percent_Change_In_Price'].apply(
    lambda x: 'Positive' if x > 0 else 'Negative'
)

NSEI['Market_Condition'] = NSEI['Percent_Change_In_Price'].apply(
    lambda x: 'Bullish' if x > 0.005 else ('Bearish' if x < -0.005 else 'Neutral')
)

In [138]:
GoldBond.head()

,Date,Series,Open,High,Low,Prev_Close,Ltp,Close,Vwap,52W_H,52W_L,Volume,Value,No_Of_Trades,Percent_Change_In_Price,Gold_Return_Category
0,2019-12-30,GB,3700.00,3700.00,3700.00,3599.50,3700.00,3700.00,3700.00,3840.0,2725.00,6,22200.00,1,NaN,Negative
1,2019-12-23,GB,3750.00,3750.00,3599.00,3589.00,3599.00,3599.50,3616.33,3840.0,2655.29,45,162735.00,5,-0.027162,Negative
2,2019-12-17,GB,3590.00,3590.00,3589.00,3580.05,3589.00,3589.00,3589.40,3840.0,2655.29,25,89735.00,3,-0.002917,Negative
3,2019-12-16,GB,3580.05,3580.05,3580.05,3580.00,3580.05,3580.05,3580.05,3840.0,2655.29,1,3580.05,1,-0.002494,Negative
4,2019-12-12,GB,3562.00,3580.00,3562.00,3585.00,3580.00,3580.00,3565.00,3840.0,2655.29,6,21390.00,2,-0.000014,Negative


In [139]:
GoldBond.tail()

,Date,Series,Open,High,Low,Prev_Close,Ltp,Close,Vwap,52W_H,52W_L,Volume,Value,No_Of_Trades,Percent_Change_In_Price,Gold_Return_Category
678,2024-01-15,GB,6240.01,6400.0,6240.01,6220.0,6400.0,6400.0,6293.34,6400.0,5360.0,6,37760.04,2,0.024000,Positive
679,2024-01-11,GB,6220.00,6220.0,6220.00,6275.0,6220.0,6220.0,6220.00,6340.0,5360.0,5,31100.00,1,-0.028125,Negative
680,2024-01-09,GB,6275.00,6275.0,6275.00,6275.0,6275.0,6275.0,6275.00,6340.0,5360.0,20,125500.00,2,0.008842,Positive
681,2024-01-08,GB,6275.00,6275.0,6275.00,6340.0,6275.0,6275.0,6275.00,6340.0,5360.0,4,25100.00,2,0.000000,Negative
682,2024-01-05,GB,6340.00,6340.0,6340.00,6340.0,6340.0,6340.0,6340.00,6340.0,5300.0,9,57060.00,2,0.010359,Positive


In [140]:
NSEI.tail()

,Date,Open,High,Low,Close,Volume,Dividends,Stock_Splits,Percent_Change_In_Price,Market_Condition
4258,2025-01-29 00:00:00+05:30,23026.750000,23183.349609,22976.500000,23163.099609,226300,0,0,0.008967,Bullish
4259,2025-01-30 00:00:00+05:30,23169.500000,23322.050781,23139.199219,23249.500000,361900,0,0,0.003730,Neutral
4260,2025-01-31 00:00:00+05:30,23296.750000,23546.800781,23277.400391,23508.400391,304900,0,0,0.011136,Bullish
4261,2025-02-03 00:00:00+05:30,23319.349609,23381.599609,23222.000000,23361.050781,329700,0,0,-0.006268,Bearish
4262,2025-02-04 00:00:00+05:30,23509.900391,23582.000000,23479.449219,23567.550781,0,0,0,0.008839,Bullish


In [141]:
NSEI['Date'] = pd.to_datetime(NSEI['Date'])
NSEI['Date'] = NSEI['Date'].dt.tz_localize(None)

In [142]:
merged_data = pd.merge(GoldBond[['Date', 'Gold_Return_Category']],
                       NSEI[['Date', 'Market_Condition']],
                       on='Date')

contingency_table = pd.crosstab(merged_data['Gold_Return_Category'], 
                                merged_data['Market_Condition'])
print(contingency_table)

Market_Condition      Bearish  Bullish  Neutral
Gold_Return_Category                           
Negative                  101      139      195
Positive                   50       77      117


In [143]:
chi2, test4_p_val, dof, expected = chi2_contingency(contingency_table)
print(f"Chi-Square Statistic: {chi2}, p-value: {test4_p_val}")

Chi-Square Statistic: 0.8621403265313069, p-value: 0.6498133161509081


In [144]:
test_hypothesis(test4_p_val)

'Fail to reject the null hypothesis.'

#### From p value of the test, we get to know that we fail to reject the null hypothesis.

#### We don't have sufficient evidences to justify that Gold bond returns are not independent of market conditions.